<a href="https://colab.research.google.com/github/Nirika-Lamichhane/Minor_Project-5-24-25-36-/blob/main/language_identification_emoji_mapped_text_cleaned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.utils import shuffle

# Load your dataset
df = pd.read_csv("/content/lang_identification_1.csv")

# Shuffle the dataset
df = shuffle(df, random_state=42).reset_index(drop=True)

# Save the shuffled dataset (optional)
df.to_csv("shuffled_dataset.csv", index=False)

# Quick check
print(df.head())
print(df['label'].value_counts())

                                                text       label
0                                Thumbnail is lit 😂😂          EN
1                  माटोलाई हाल्ने हो जितोस या हारोस्      NE_DEV
2          nicholous❌ चाँडै (निको होस) mental cramps  CODE_MIXED
3                                      He is right ❤          EN
4  Twist ta Aayo ta kulman neskeya aaba raja aaun...      NE_ROM
label
EN            496
NE_ROM        385
NE_DEV        275
CODE_MIXED    214
Name: count, dtype: int64


In [2]:
# Step 1: Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# Step 2: Load your shuffled dataset
# Assuming your dataset is CSV with two columns: 'text' and 'label'
df = pd.read_csv('/content/lang_identification_1.csv')  # replace with your CSV path

texts = df['text'].astype(str).tolist()
labels = df['label'].tolist()

# Step 3: Split into training and testing sets (80%-20%)
X_train, X_test, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)

# Step 4: TF-IDF vectorization
tfidf_vectorizer = TfidfVectorizer(
    analyzer='char_wb',   # character n-grams help for short Roman and noisy text
    ngram_range=(2, 5),   # 2-5 char ngrams
    lowercase=True,
    max_features=10000    # limit features to keep it manageable
)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Step 5: Train SVM
svm_clf = SVC(
    kernel='linear',   # linear kernel works well for text
    C=1.0,             # regularization parameter
    probability=True,  # allows probability output if needed
    class_weight='balanced',  # handle class imbalance
    random_state=42
)

svm_clf.fit(X_train_tfidf, y_train)

# Step 6: Evaluate
y_pred = svm_clf.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.8832116788321168

Classification Report:
               precision    recall  f1-score   support

  CODE_MIXED       0.76      0.58      0.66        43
          EN       0.87      0.97      0.92        99
      NE_DEV       0.96      1.00      0.98        55
      NE_ROM       0.89      0.86      0.87        77

    accuracy                           0.88       274
   macro avg       0.87      0.85      0.86       274
weighted avg       0.88      0.88      0.88       274



In [8]:
!pip install numpy==1.26.4



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 95.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytensor 2.36.3 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
rasterio 1.5.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; pyth

In [3]:
pip install fasttext --upgrade


In [ ]:
import fasttext


In [4]:
import fasttext
import pandas as pd
from sklearn.model_selection import train_test_split

# 1. Load your dataset (CSV with columns: text, label)
df = pd.read_csv("/content/lang_identification_1.csv")

# 2. Convert to FastText format
# FastText expects each line as: __label__<label> <text>
df['fasttext_format'] = "__label__" + df['label'] + " " + df['text']

# 3. Split into training and validation sets
train_texts, val_texts = train_test_split(df['fasttext_format'], test_size=0.2, random_state=42, shuffle=True)

# 4. Save the datasets to temporary text files for FastText
with open("train.txt", "w", encoding="utf-8") as f:
    for line in train_texts:
        f.write(line + "\n")

with open("valid.txt", "w", encoding="utf-8") as f:
    for line in val_texts:
        f.write(line + "\n")

# 5. Train the supervised FastText model
model = fasttext.train_supervised(
    input="train.txt",
    lr=0.5,              # learning rate, higher value for faster convergence
    epoch=50,            # number of training iterations over the dataset
    wordNgrams=2,        # use bigrams for capturing context (helps code-mixed)
    dim=300,             # size of word vectors
    minn=2, maxn=5,      # subword information for OOV words and Roman Nepali
    bucket=200000,       # hash bucket size for subwords
    loss='softmax',      # softmax is standard for multi-class classification
    thread=4,            # number of CPU threads to use
)

# 6. Evaluate on validation set
result = model.test("valid.txt")
print("FastText Validation Metrics:")
print(f"Samples: {result[0]}, Precision: {result[1]:.4f}, Recall: {result[2]:.4f}")

# 7. Save the trained model for later use
model.save_model("fasttext_language_model.bin")




FastText Validation Metrics:
Samples: 274, Precision: 0.8905, Recall: 0.8905


In [10]:
import fasttext
import re
import numpy as np

# Load FastText model
ft_model = fasttext.load_model("fasttext_language_model.bin")

# --- Helper: text cleaning ---
def clean_text(text):
    text = str(text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

# --- Word count ---
def word_count(text):
    return len(text.split())

# --- SVM prediction ---
def predict_svm(text):
    text_tfidf = tfidf_vectorizer.transform([text])
    pred = svm_clf.predict(text_tfidf)[0]
    prob = max(svm_clf.predict_proba(text_tfidf)[0])
    return pred, prob

# --- FastText prediction ---
def predict_fasttext(text):
    labels, probs = ft_model.predict(text)

    # Force safe conversion (NumPy 2.x compatible)
    labels = list(labels)
    probs = np.asarray(probs)

    label = labels[0].replace("__label__", "")
    confidence = float(probs[0])

    return label, confidence

# --- Hybrid prediction ---

CONFIDENCE_THRESHOLD = 0.5
def hybrid_language_detect(text, threshold=6):
    text = clean_text(text)
    wc = word_count(text)

    if wc <= threshold:
        label, confidence = predict_svm(text)
        source = "SVM"
    else:
        label, confidence = predict_fasttext(text)
        source = "FastText"

    if confidence < CONFIDENCE_THRESHOLD or wc < 2:
        return {
            "text": text,
            "predicted_label": "UNKNOWN",
            "confidence": round(float(confidence), 4),
            "model_used": source,
            "word_count": wc
        }

    return {
        "text": text,
        "predicted_label": label,
        "confidence": round(confidence, 4),
        "model_used": source,
        "word_count": wc
    }


In [11]:
examples = [
    "kp chor 👍",
    "सबै व्यक्तिहरू जन्मजात स्वतन्त्र हुन्",
    "lau yaar not true",
    "vote for balen now or never 🇳🇵",
    "Jhapa ma aunu parcha",
    "xya"
]

for e in examples:
    print(hybrid_language_detect(e))


{'text': 'kp chor 👍', 'predicted_label': 'EN', 'confidence': 0.6531, 'model_used': 'SVM', 'word_count': 3}
{'text': 'सबै व्यक्तिहरू जन्मजात स्वतन्त्र हुन्', 'predicted_label': 'NE_DEV', 'confidence': 0.9779, 'model_used': 'SVM', 'word_count': 5}
{'text': 'lau yaar not true', 'predicted_label': 'CODE_MIXED', 'confidence': 0.7713, 'model_used': 'SVM', 'word_count': 4}
{'text': 'vote for balen now or never 🇳🇵', 'predicted_label': 'EN', 'confidence': 0.9994, 'model_used': 'FastText', 'word_count': 7}
{'text': 'Jhapa ma aunu parcha', 'predicted_label': 'NE_ROM', 'confidence': 0.891, 'model_used': 'SVM', 'word_count': 4}
{'text': 'xya', 'predicted_label': 'UNKNOWN', 'confidence': 0.736, 'model_used': 'SVM', 'word_count': 1}


In [12]:
hybrid_language_detect("yo sarkar ramrooooo cha <EMOJI>")

{'text': 'yo sarkar ramrooooo cha <EMOJI>',
 'predicted_label': 'NE_ROM',
 'confidence': 0.519,
 'model_used': 'SVM',
 'word_count': 5}

In [83]:
comments = [
    "बालेन 😂राम्रै छ!!!! 😂🔥",
    "This government is corrupt #murderer 😡",
    "ramro kaam ho 'https://www.youtube.com/watch?v=lsV8JQgSW1s&list=PLu0W_9lII9ahR1blWXxgSlL4y9iQBnLpR&index=6' ❤️",
    "नेपाल जित्छ    💪🇳🇵",
    "Worst service ever...... 😠👎"
]


In [84]:
import pandas as pd

rows = []

for comment in comments:
    result = hybrid_language_detect(comment)

    rows.append({
        "raw_text": comment,
        "detected_language": result["predicted_label"],
        "confidence": result["confidence"],
        "model_used": result["model_used"],
        "word_count": result["word_count"]
    })

df = pd.DataFrame(rows)
df


,raw_text,detected_language,confidence,model_used,word_count
0,बालेन 😂राम्रै छ!!!! 😂🔥,NE_DEV,0.9609,SVM,3
1,This government is corrupt #murderer 😡,EN,0.9348,SVM,4
2,ramro kaam ho 'https://www.youtube.com/watch?v...,NE_ROM,0.6750,SVM,3
3,नेपाल जित्छ 💪🇳🇵,NE_DEV,0.8058,SVM,2
4,Worst service ever...... 😠👎,EN,0.9743,SVM,3


In [85]:
import re

EMOJI_PATTERN = re.compile(
    "["
    "\U0001F600-\U0001F64F"  # emoticons
    "\U0001F300-\U0001F5FF"  # symbols & pictographs
    "\U0001F680-\U0001F6FF"  # transport & map
    "\U0001F1E0-\U0001F1FF"  # flags
    "\U00002700-\U000027BF"
    "]",
    flags=re.UNICODE
)

def extract_emojis(text):
    return EMOJI_PATTERN.findall(text)


In [86]:
df["emoji_list"] = df["raw_text"].apply(extract_emojis)
df


,raw_text,detected_language,confidence,model_used,word_count,emoji_list
0,बालेन 😂राम्रै छ!!!! 😂🔥,NE_DEV,0.9609,SVM,3,"[😂, 😂, 🔥]"
1,This government is corrupt #murderer 😡,EN,0.9348,SVM,4,[😡]
2,ramro kaam ho 'https://www.youtube.com/watch?v...,NE_ROM,0.6750,SVM,3,[❤]
3,नेपाल जित्छ 💪🇳🇵,NE_DEV,0.8058,SVM,2,"[💪, 🇳, 🇵]"
4,Worst service ever...... 😠👎,EN,0.9743,SVM,3,"[😠, 👎]"


In [87]:
EMOJI_SENTIMENT_MAP = {
    # Positive
    "😂": "positive",
    "🤣": "positive",
    "😄": "positive",
    "😊": "positive",
    "❤️": "positive",
    "🔥": "positive",
    "💪": "positive",
    "👍": "positive",

    # Negative
    "😡": "negative",
    "😠": "negative",
    "👎": "negative",
    "💔": "negative",
    "😭": "negative",

    # Neutral / discourse
    "🤔": "neutral",
    "😐": "neutral",
    "🫠": "neutral",

    # Sarcasm / ambiguous
    "🙃": "sarcasm",
    "😏": "sarcasm",
}



In [88]:
from collections import Counter

def emoji_feature_vector(emoji_list):
    counts = Counter()

    for e in emoji_list:
        category = EMOJI_SENTIMENT_MAP.get(e, "unknown")
        counts[category] += 1

    return {
        "emoji_pos_count": counts.get("positive", 0),
        "emoji_neg_count": counts.get("negative", 0),
        "emoji_neu_count": counts.get("neutral", 0),
        "emoji_sarcasm_count": counts.get("sarcasm", 0),
        "emoji_unknown_count": counts.get("unknown", 0),
        "emoji_total": sum(counts.values())
    }


In [89]:
emoji_features = df["emoji_list"].apply(emoji_feature_vector)
emoji_df = pd.DataFrame(emoji_features.tolist())

df = pd.concat([df, emoji_df], axis=1)


In [90]:
def dominant_emoji_sentiment(row):
    if row["emoji_total"] == 0:
        return "none"

    if row["emoji_neg_count"] > row["emoji_pos_count"]:
        return "negative"

    if row["emoji_pos_count"] > row["emoji_neg_count"]:
        return "positive"

    if row["emoji_sarcasm_count"] > 0:
        return "sarcasm"

    return "mixed"


In [91]:
df["emoji_dominant_sentiment"] = df.apply(dominant_emoji_sentiment, axis=1)


In [92]:
df

,raw_text,detected_language,confidence,model_used,word_count,emoji_list,emoji_pos_count,emoji_neg_count,emoji_neu_count,emoji_sarcasm_count,emoji_unknown_count,emoji_total,emoji_dominant_sentiment
0,बालेन 😂राम्रै छ!!!! 😂🔥,NE_DEV,0.9609,SVM,3,"[😂, 😂, 🔥]",3,0,0,0,0,3,positive
1,This government is corrupt #murderer 😡,EN,0.9348,SVM,4,[😡],0,1,0,0,0,1,negative
2,ramro kaam ho 'https://www.youtube.com/watch?v...,NE_ROM,0.6750,SVM,3,[❤],0,0,0,0,1,1,mixed
3,नेपाल जित्छ 💪🇳🇵,NE_DEV,0.8058,SVM,2,"[💪, 🇳, 🇵]",1,0,0,0,2,3,positive
4,Worst service ever...... 😠👎,EN,0.9743,SVM,3,"[😠, 👎]",0,2,0,0,0,2,negative


In [93]:
def remove_emojis(text):
    return EMOJI_PATTERN.sub("", text).strip()

df["text_no_emoji"] = df["raw_text"].apply(remove_emojis)



In [73]:
df

,raw_text,detected_language,confidence,model_used,word_count,emoji_list,emoji_pos_count,emoji_neg_count,emoji_neu_count,emoji_sarcasm_count,emoji_unknown_count,emoji_total,emoji_dominant_sentiment,text_no_emoji
0,बालेन 😂राम्रै छ 😂🔥,NE_DEV,0.9405,SVM,4,"[😂, 😂, 🔥]",3,0,0,0,0,3,positive,बालेन राम्रै छ
1,This government is corrupt 😡,EN,0.9251,SVM,5,[😡],0,1,0,0,0,1,negative,This government is corrupt
2,ramro kaam ho ❤️,NE_ROM,0.6923,SVM,4,[❤],0,0,0,0,1,1,mixed,ramro kaam ho ️
3,नेपाल जित्छ 💪🇳🇵,NE_DEV,0.8523,SVM,3,"[💪, 🇳, 🇵]",1,0,0,0,2,3,positive,नेपाल जित्छ
4,Worst service ever 😠👎,EN,0.9743,SVM,4,"[😠, 👎]",0,2,0,0,0,2,negative,Worst service ever


In [94]:
# Keep only the necessary columns
necessary_columns = [
    "raw_text",
    "text_no_emoji",
    "emoji_list",
    "emoji_dominant_sentiment",
    "detected_language"
]

df = df[necessary_columns]


In [95]:
df

,raw_text,text_no_emoji,emoji_list,emoji_dominant_sentiment,detected_language
0,बालेन 😂राम्रै छ!!!! 😂🔥,बालेन राम्रै छ!!!!,"[😂, 😂, 🔥]",positive,NE_DEV
1,This government is corrupt #murderer 😡,This government is corrupt #murderer,[😡],negative,EN
2,ramro kaam ho 'https://www.youtube.com/watch?v...,ramro kaam ho 'https://www.youtube.com/watch?v...,[❤],mixed,NE_ROM
3,नेपाल जित्छ 💪🇳🇵,नेपाल जित्छ,"[💪, 🇳, 🇵]",positive,NE_DEV
4,Worst service ever...... 😠👎,Worst service ever......,"[😠, 👎]",negative,EN


In [96]:
import re

URL_PATTERN = re.compile(r"https?://\S+|www\.\S+")
MENTION_PATTERN = re.compile(r"@\w+")
HASHTAG_PATTERN = re.compile(r"#\w+")

# Keep: Devanagari + Roman + space
NON_TEXT_PATTERN = re.compile(r"[^a-zA-Z\u0900-\u097F\s]")

MULTISPACE_PATTERN = re.compile(r"\s+")


def clean_text(text):
    text = text.lower()

    # Remove URLs
    text = URL_PATTERN.sub("", text)

    # Remove mentions
    text = MENTION_PATTERN.sub("", text)

    # Remove hashtags completely
    text = HASHTAG_PATTERN.sub("", text)

    # Remove punctuation & symbols but KEEP Devanagari
    text = NON_TEXT_PATTERN.sub(" ", text)

    # Normalize spaces
    text = MULTISPACE_PATTERN.sub(" ", text)

    return text.strip()


In [97]:
df["text_no_emoji"]


,text_no_emoji
0,बालेन राम्रै छ!!!!
1,This government is corrupt #murderer
2,ramro kaam ho 'https://www.youtube.com/watch?v...
3,नेपाल जित्छ
4,Worst service ever......


In [98]:
df["clean_text"] = df["text_no_emoji"].apply(clean_text)


/tmp/ipython-input-2490990152.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["clean_text"] = df["text_no_emoji"].apply(clean_text)


In [99]:
df

,raw_text,text_no_emoji,emoji_list,emoji_dominant_sentiment,detected_language,clean_text
0,बालेन 😂राम्रै छ!!!! 😂🔥,बालेन राम्रै छ!!!!,"[😂, 😂, 🔥]",positive,NE_DEV,बालेन राम्रै छ
1,This government is corrupt #murderer 😡,This government is corrupt #murderer,[😡],negative,EN,this government is corrupt
2,ramro kaam ho 'https://www.youtube.com/watch?v...,ramro kaam ho 'https://www.youtube.com/watch?v...,[❤],mixed,NE_ROM,ramro kaam ho
3,नेपाल जित्छ 💪🇳🇵,नेपाल जित्छ,"[💪, 🇳, 🇵]",positive,NE_DEV,नेपाल जित्छ
4,Worst service ever...... 😠👎,Worst service ever......,"[😠, 👎]",negative,EN,worst service ever


**to be continued**.........

In [15]:
import re

# --- Step 1: Token-level filtering for transliteration ---
ENGLISH_STOPWORDS = set([
    "the", "is", "are", "was", "were", "in", "on", "at", "and", "or", "but",
    "this", "that", "with", "by", "from", "your", "you", "for", "of", "a", "an",
    "to", "government", "policy", "road", "house"  # extend with domain-specific common English words
])

DEVANAGARI_REGEX = re.compile(r'[\u0900-\u097F]')  # matches Devanagari chars
NUMERIC_REGEX = re.compile(r'^\d+$')               # matches pure numbers

def token_level_filter(text, predicted_label):
    """
    Inputs:
        text: raw comment string
        predicted_label: output of language detection (Nepali_Roman, English, etc.)
    Outputs:
        list of tuples: (token, action) where action = 'TRANSLITERATE' or 'SKIP'
    """
    filtered_tokens = []

    # Only process Nepali Roman for transliteration
    if predicted_label != "NE_ROM":
        return []

    # Step 1: basic tokenization (whitespace)
    tokens = text.split()

    for token in tokens:
        token_lower = token.lower()

        # Rule 1: Skip special tokens
        if token in ["<EMOJI>", "<URL>", "<NUM>"]:
            filtered_tokens.append((token, "SKIP"))
            continue

        # Rule 2: Skip Devanagari words
        if DEVANAGARI_REGEX.search(token):
            filtered_tokens.append((token, "SKIP"))
            continue

        # Rule 3: Skip numeric tokens
        if NUMERIC_REGEX.match(token):
            filtered_tokens.append((token, "SKIP"))
            continue

        # Rule 4: Skip short tokens (length < 2)
        if len(token) < 2:
            filtered_tokens.append((token, "SKIP"))
            continue

        # Rule 5: Skip English stopwords / common English words
        if token_lower in ENGLISH_STOPWORDS:
            filtered_tokens.append((token, "SKIP"))
            continue

        # Rule 6: Remaining tokens -> candidate for transliteration
        filtered_tokens.append((token, "TRANSLITERATE"))

    return filtered_tokens

# --- Example usage after language detection ---

comments = [
    "yo sarkar ramrooooo cha <EMOJI>",
    "your government is strong",
    "ramro desh ho"
]

for comment in comments:
    detection_result = hybrid_language_detect(comment)  # using your existing function
    filtered_tokens = token_level_filter(detection_result['text'], detection_result['predicted_label'])
    print(f"Original: {comment}")
    print(f"Filtered Tokens: {filtered_tokens}\n")
    print(token_level_filter(detection_result['text'], detection_result['predicted_label']))


Original: yo sarkar ramrooooo cha <EMOJI>😁
Filtered Tokens: [('yo', 'TRANSLITERATE'), ('sarkar', 'TRANSLITERATE'), ('ramrooooo', 'TRANSLITERATE'), ('cha', 'TRANSLITERATE'), ('<EMOJI>😁', 'TRANSLITERATE')]

[('yo', 'TRANSLITERATE'), ('sarkar', 'TRANSLITERATE'), ('ramrooooo', 'TRANSLITERATE'), ('cha', 'TRANSLITERATE'), ('<EMOJI>😁', 'TRANSLITERATE')]
Original: your government is strong
Filtered Tokens: []

[]
Original: ramro desh ho
Filtered Tokens: [('ramro', 'TRANSLITERATE'), ('desh', 'TRANSLITERATE'), ('ho', 'TRANSLITERATE')]

[('ramro', 'TRANSLITERATE'), ('desh', 'TRANSLITERATE'), ('ho', 'TRANSLITERATE')]


In [ ]:
import os

base = "/content/drive/MyDrive/Colab Notebooks/LanguageID"
os.makedirs(base + "/data", exist_ok=True)
os.makedirs(base + "/models", exist_ok=True)
os.makedirs(base + "/outputs", exist_ok=True)


In [ ]:
df.to_csv("/content/drive/MyDrive/Colab Notebooks/LanguageID/data/processed.csv", index=False)

